<a href="https://colab.research.google.com/github/wa-le/BE-Properties-Regression/blob/master/be_properties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Goal

The aim is to find patterns in the [Belgian Property Prices 2023](https://www.kaggle.com/datasets/unworried1686/belgian-property-prices-2023) dataset and build a model to predict prices.


In [ ]:
#%pip install pycaret

In [3]:
# change working directory
%cd /content/drive/MyDrive/Data Projects/Belgium Properties

/content/drive/MyDrive/Data Projects/Belgium Properties


In [4]:
%pwd

'/content/drive/MyDrive/Data Projects/Belgium Properties'

# Imports and Load dataset

In [117]:
# Data Analysis
import pandas as pd
import numpy as np

# check null values in DF
import missingno

# Charts
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

# ML libraries
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV

In [6]:
# unzip file containing dataset
%%python -c "import zipfile; zip_ref = zipfile.ZipFile('belgian_property_prices.csv.zip', 'r'); zip_ref.extractall('/content/drive/MyDrive/Data Projects/Belgium Properties'); zip_ref.close()"

In [7]:
# Load CSV file
housing_df = pd.read_csv("/content/drive/MyDrive/Data Projects/Belgium Properties/belgian_property_prices.csv")
housing_df.shape

(4770, 54)

In [8]:
housing_df.head(5)

,as_built_plan,available_as_of,basement,bathrooms,bedroom_1_surface,bedroom_2_surface,bedroom_3_surface,bedrooms,building_condition,co2_emission,...,width_of_the_lot_on_the_street,yearly_theoretical_total_energy_consumption,housenumber,street,city,postal,state,lat,lng,price
0,0.0,After signing the deed,1.0,1.0,12.0,10.0,10.0,3.0,Good,9802.0,...,11.0,NaN,30,Stationsstraat,Ronse,9600.0,Vlaams Gewest,50.743367,3.601306,275000
1,0.0,After signing the deed,1.0,1.0,15.0,11.0,11.0,4.0,Good,NaN,...,NaN,NaN,52,Wingenesteenweg,Wingene,8750.0,Vlaams Gewest,51.042146,3.302600,430000
2,NaN,After signing the deed,1.0,3.0,22.0,22.0,22.0,5.0,As new,NaN,...,6.0,NaN,157,Mechelsesteenweg,Antwerpen,2018.0,Vlaams Gewest,51.205030,4.410943,949000
3,0.0,NaN,NaN,3.0,33.0,30.0,23.0,5.0,As new,NaN,...,19.0,NaN,66,Noordzandstraat,Brugge,8000.0,Vlaams Gewest,51.206826,3.219186,2200000
4,0.0,After signing the deed,NaN,NaN,19.0,NaN,NaN,1.0,Just renovated,58.0,...,5.0,24263.0,15,Rue de l'Etoile,Wavre,1301.0,Région Wallonne,50.717062,4.577550,195000


In [9]:
housing_df.head(5).T

,0,1,2,3,4
as_built_plan,0.0,0.0,NaN,0.0,0.0
available_as_of,After signing the deed,After signing the deed,After signing the deed,NaN,After signing the deed
basement,1.0,1.0,1.0,NaN,NaN
bathrooms,1.0,1.0,3.0,3.0,NaN
bedroom_1_surface,12.0,15.0,22.0,33.0,19.0
bedroom_2_surface,10.0,11.0,22.0,30.0,NaN
bedroom_3_surface,10.0,11.0,22.0,23.0,NaN
bedrooms,3.0,4.0,5.0,5.0,1.0
building_condition,Good,Good,As new,As new,Just renovated
co2_emission,9802.0,NaN,NaN,NaN,58.0


# Explore Data

In [10]:
# drop duplicates
housing_df = housing_df.drop_duplicates()
housing_df

,as_built_plan,available_as_of,basement,bathrooms,bedroom_1_surface,bedroom_2_surface,bedroom_3_surface,bedrooms,building_condition,co2_emission,...,width_of_the_lot_on_the_street,yearly_theoretical_total_energy_consumption,housenumber,street,city,postal,state,lat,lng,price
0,0.0,After signing the deed,1.0,1.0,12.0,10.0,10.0,3.0,Good,9802.0,...,11.0,NaN,30,Stationsstraat,Ronse,9600.0,Vlaams Gewest,50.743367,3.601306,275000
1,0.0,After signing the deed,1.0,1.0,15.0,11.0,11.0,4.0,Good,NaN,...,NaN,NaN,52,Wingenesteenweg,Wingene,8750.0,Vlaams Gewest,51.042146,3.302600,430000
2,NaN,After signing the deed,1.0,3.0,22.0,22.0,22.0,5.0,As new,NaN,...,6.0,NaN,157,Mechelsesteenweg,Antwerpen,2018.0,Vlaams Gewest,51.205030,4.410943,949000
3,0.0,NaN,NaN,3.0,33.0,30.0,23.0,5.0,As new,NaN,...,19.0,NaN,66,Noordzandstraat,Brugge,8000.0,Vlaams Gewest,51.206826,3.219186,2200000
4,0.0,After signing the deed,NaN,NaN,19.0,NaN,NaN,1.0,Just renovated,58.0,...,5.0,24263.0,15,Rue de l'Etoile,Wavre,1301.0,Région Wallonne,50.717062,4.577550,195000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4765,1.0,After signing the deed,1.0,2.0,11.0,14.0,15.0,6.0,Good,NaN,...,NaN,NaN,594,Brusselsesteenweg,Asse,1731.0,Vlaams Gewest,50.884192,4.274010,944000
4766,NaN,After signing the deed,1.0,2.0,25.0,20.0,18.0,6.0,To be done up,NaN,...,NaN,NaN,31,Brusselsesteenweg,Zemst,1980.0,Vlaams Gewest,50.985378,4.466114,960000
4767,0.0,NaN,1.0,5.0,NaN,NaN,NaN,9.0,Good,NaN,...,NaN,129394.0,15,Rue de Nazareth,Ath,7800.0,Région Wallonne,50.630171,3.779321,750000
4768,0.0,To be defined,1.0,2.0,NaN,NaN,NaN,3.0,Good,NaN,...,9.0,NaN,8,Orteliuskaai,Antwerpen,2000.0,Vlaams Gewest,51.225206,4.399929,925000


In [11]:
# checking each columns datatype
housing_df.dtypes

as_built_plan                                     float64
available_as_of                                    object
basement                                          float64
bathrooms                                         float64
bedroom_1_surface                                 float64
bedroom_2_surface                                 float64
bedroom_3_surface                                 float64
bedrooms                                          float64
building_condition                                 object
co2_emission                                      float64
cadastral_income                                  float64
connection_to_sewer_network                       float64
construction_year                                 float64
covered_parking_spaces                            float64
dining_room                                       float64
double_glazing                                    float64
energy_class                                       object
external_refer

In [12]:
# checking more info on our data
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4753 entries, 0 to 4769
Data columns (total 54 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   as_built_plan                                   2381 non-null   float64
 1   available_as_of                                 3899 non-null   object 
 2   basement                                        2073 non-null   float64
 3   bathrooms                                       4507 non-null   float64
 4   bedroom_1_surface                               3150 non-null   float64
 5   bedroom_2_surface                               3093 non-null   float64
 6   bedroom_3_surface                               2611 non-null   float64
 7   bedrooms                                        4723 non-null   float64
 8   building_condition                              4470 non-null   object 
 9   co2_emission                             

In [13]:
# General statistics on dataset
housing_df.describe().T

,count,mean,std,min,25%,50%,75%,max
as_built_plan,2381.0,0.061319,0.239965,0.000000,0.000000,0.000000,0.000000,1.000000e+00
basement,2073.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00
bathrooms,4507.0,1.619481,1.109057,1.000000,1.000000,1.000000,2.000000,1.300000e+01
bedroom_1_surface,3150.0,18.344444,27.286463,1.000000,14.000000,16.000000,20.000000,1.492000e+03
bedroom_2_surface,3093.0,16.167798,36.615109,1.000000,12.000000,14.000000,17.000000,1.529000e+03
bedroom_3_surface,2611.0,17.792800,79.881629,1.000000,11.000000,14.000000,17.000000,2.867000e+03
bedrooms,4723.0,3.745924,1.528157,1.000000,3.000000,3.000000,4.000000,2.500000e+01
co2_emission,1927.0,539.125065,2199.065293,1.000000,49.000000,76.000000,116.000000,5.655700e+04
cadastral_income,3679.0,1364.385431,5682.025996,1.000000,567.000000,930.000000,1506.000000,3.369650e+05
connection_to_sewer_network,2021.0,0.927759,0.258951,0.000000,1.000000,1.000000,1.000000,1.000000e+00


In [14]:
housing_df.columns

Index(['as_built_plan', 'available_as_of', 'basement', 'bathrooms',
       'bedroom_1_surface', 'bedroom_2_surface', 'bedroom_3_surface',
       'bedrooms', 'building_condition', 'co2_emission', 'cadastral_income',
       'connection_to_sewer_network', 'construction_year',
       'covered_parking_spaces', 'dining_room', 'double_glazing',
       'energy_class', 'external_reference', 'flood_zone_type', 'furnished',
       'garden_surface', 'gas_water__electricity', 'heating_type',
       'kitchen_surface', 'kitchen_type', 'latest_land_use_designation',
       'living_area', 'living_room_surface', 'number_of_frontages', 'office',
       'outdoor_parking_spaces', 'planning_permission_obtained',
       'possible_priority_purchase_right', 'primary_energy_consumption',
       'proceedings_for_breach_of_planning_regulations',
       'reference_number_of_the_epc_report', 'street_frontage_width',
       'subdivision_permit', 'surface_of_the_plot', 'surroundings_type',
       'tv_cable', 'tenemen

In [15]:
# Select columns that contain only string values
str_cols = housing_df.select_dtypes(include=['object'])

# Check the unique values in the columns
for each in str_cols.columns:
  print(each)
  print(housing_df[each].value_counts().T)
  print("")

available_as_of
After signing the deed     2900
Immediately                 376
To be defined               341
Depending on the tenant     198
At delivery                  84
Name: available_as_of, dtype: int64

building_condition
Good              1959
As new             783
To renovate        693
To be done up      644
Just renovated     321
To restore          70
Name: building_condition, dtype: int64

energy_class
D                877
C                812
F                799
E                742
B                663
G                482
Not specified    193
A                171
A+                11
A++                2
C_B                1
Name: energy_class, dtype: int64

external_reference
D1050-22320          2
5596472              1
5513866              1
5525227              1
5251923              1
                    ..
11553 - 4129         1
5528349              1
4975 - 2459956       1
11715 - 111114987    1
1165 - 474           1
Name: external_reference, Length: 4319, 

In [16]:
housing_df.head(1).T

,0
as_built_plan,0.0
available_as_of,After signing the deed
basement,1.0
bathrooms,1.0
bedroom_1_surface,12.0
bedroom_2_surface,10.0
bedroom_3_surface,10.0
bedrooms,3.0
building_condition,Good
co2_emission,9802.0


### Feature Engineering

In [17]:
# change 'energy_class' value C_B to C
housing_df['energy_class'] = np.where(housing_df['energy_class'] == 'C_B', 'C', housing_df['energy_class'])

In [18]:
# fix values in 'kitchen_type'
housing_df['kitchen_type'] = np.where(housing_df['kitchen_type'] == 'USA uninstalled', 'USA installed', housing_df['kitchen_type'])

In [19]:
housing_df["kitchen_type"].value_counts()

Installed             1734
Semi equipped          818
Hyper equipped         629
USA hyper equipped     297
Not installed          283
USA installed          155
USA semi equipped       31
Name: kitchen_type, dtype: int64

In [20]:
housing_df["city"].value_counts()

Seraing       193
Antwerpen     159
Gent          142
Liège         141
Huy           140
             ... 
Wijnegem        1
Seneffe         1
Herzele         1
Liedekerke      1
Zemst           1
Name: city, Length: 246, dtype: int64

In [21]:
housing_df["state"].value_counts()

Vlaams Gewest      2094
Région Wallonne    1946
Bruxelles           571
Waals Gewest         54
Wallonia             19
Flanders             17
Brussels              6
Brussel               2
Région Flamande       1
Capellen              1
Name: state, dtype: int64

In [22]:
# fix values in 'state'
housing_df['state'] = np.where(housing_df['state'] == 'Région Wallonne', 'Wallonia', housing_df['state'])
housing_df['state'] = np.where(housing_df['state'] == 'Bruxelles', 'Brussel', housing_df['state'])
housing_df['state'] = np.where(housing_df['state'] == 'Brussels', 'Brussel', housing_df['state'])
housing_df['state'] = np.where(housing_df['state'] == 'Région Flamande', 'Flanders', housing_df['state'])

In [23]:
# Capelen state is not in Belgium, deleted the row
housing_df = housing_df[~(housing_df["state"] == "Capellen")]

In [24]:
housing_df["state"].value_counts()

Vlaams Gewest    2094
Wallonia         1965
Brussel           579
Waals Gewest       54
Flanders           18
Name: state, dtype: int64

In [25]:
# still trying to fix the states - not sure of waals geweest
housing_df[housing_df["state"] == "Waals Gewest"]

#checked few cities in waals geweest and it seems its still Wallonia
housing_df['state'] = np.where(housing_df['state'] == 'Waals Gewest', 'Wallonia', housing_df['state'])

<ipython-input-25-b2896f8a109a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_df['state'] = np.where(housing_df['state'] == 'Waals Gewest', 'Wallonia', housing_df['state'])


In [26]:
# will do same for Vlaams Gewest
housing_df[housing_df["state"] == "Waals Gewest"]
housing_df['state'] = np.where(housing_df['state'] == 'Vlaams Gewest', 'Flanders', housing_df['state'])

<ipython-input-26-c7977954ddcc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_df['state'] = np.where(housing_df['state'] == 'Vlaams Gewest', 'Flanders', housing_df['state'])


# Modelling

## Model-Driven EDA
  - Take care of Null Values
  - Convert non numeric to numeric


In [27]:
housing_df.isna().sum()

as_built_plan                                     2371
available_as_of                                    854
basement                                          2680
bathrooms                                          246
bedroom_1_surface                                 1602
bedroom_2_surface                                 1659
bedroom_3_surface                                 2141
bedrooms                                            30
building_condition                                 283
co2_emission                                      2826
cadastral_income                                  1073
connection_to_sewer_network                       2732
construction_year                                 1317
covered_parking_spaces                            2043
dining_room                                       3419
double_glazing                                     649
energy_class                                         0
external_reference                                 432
flood_zone

In [28]:
housing_df.head(5).T

,0,1,2,3,4
as_built_plan,0.0,0.0,NaN,0.0,0.0
available_as_of,After signing the deed,After signing the deed,After signing the deed,NaN,After signing the deed
basement,1.0,1.0,1.0,NaN,NaN
bathrooms,1.0,1.0,3.0,3.0,NaN
bedroom_1_surface,12.0,15.0,22.0,33.0,19.0
bedroom_2_surface,10.0,11.0,22.0,30.0,NaN
bedroom_3_surface,10.0,11.0,22.0,23.0,NaN
bedrooms,3.0,4.0,5.0,5.0,1.0
building_condition,Good,Good,As new,As new,Just renovated
co2_emission,9802.0,NaN,NaN,NaN,58.0


In [29]:
# trying to fill missing lat and lng
rows_with_null = housing_df[housing_df["lat"].isnull()]
rows_with_null[["housenumber","street", "city", "postal", "state", "lat", "lng"]]

# found out all possible info I can use to fill missing lat and lng is also missing
# Then I decided to drop such columns

,housenumber,street,city,postal,state,lat,lng
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597,NaN,NaN,NaN,NaN,NaN,NaN,NaN
611,NaN,NaN,NaN,NaN,NaN,NaN,NaN
822,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1283,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1751,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1754,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2208,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2252,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# Then I decided to remove such columns
housing_df = housing_df[~housing_df["lat"].isnull()]

In [31]:
housing_df.isna().sum()

as_built_plan                                     2340
available_as_of                                    850
basement                                          2655
bathrooms                                          243
bedroom_1_surface                                 1590
bedroom_2_surface                                 1647
bedroom_3_surface                                 2127
bedrooms                                            30
building_condition                                 283
co2_emission                                      2802
cadastral_income                                  1068
connection_to_sewer_network                       2724
construction_year                                 1311
covered_parking_spaces                            2018
dining_room                                       3400
double_glazing                                     649
energy_class                                         0
external_reference                                 399
flood_zone

In [32]:
# Convert [postal, lat, lng] features to string(object)
# p-iter task

housing_df["postal"] = housing_df["postal"].astype(str)
housing_df["lng"] = housing_df["lng"].astype(str)
housing_df["lat"] = housing_df["lat"].astype(str)
housing_df.dtypes

as_built_plan                                     float64
available_as_of                                    object
basement                                          float64
bathrooms                                         float64
bedroom_1_surface                                 float64
bedroom_2_surface                                 float64
bedroom_3_surface                                 float64
bedrooms                                          float64
building_condition                                 object
co2_emission                                      float64
cadastral_income                                  float64
connection_to_sewer_network                       float64
construction_year                                 float64
covered_parking_spaces                            float64
dining_room                                       float64
double_glazing                                    float64
energy_class                                       object
external_refer

### Convert strings to categories

In [33]:
from pandas.api.types import is_string_dtype as is_str

In [34]:
display(is_str(housing_df["available_as_of"]))
display(is_str(housing_df["bathrooms"]))

True

False

In [35]:
is_str(housing_df["bathrooms"])

False

In [36]:
# find columns with string dtypes
for label, content in housing_df.items():
  if is_str(content):
    print(label)

available_as_of
building_condition
energy_class
external_reference
heating_type
kitchen_type
latest_land_use_designation
reference_number_of_the_epc_report
surroundings_type
website
housenumber
street
city
postal
state
lat
lng


In [37]:
# Turn string values into category

# loop through df
for label, content in housing_df.items():
  # using the pd api types lib - check if col is string
  if is_str(content):
    # if col is string, convert the col to category
    housing_df[label] = content.astype("category").cat.as_ordered()

In [38]:
# Strings now turned into categories
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4719 entries, 0 to 4769
Data columns (total 54 columns):
 #   Column                                          Non-Null Count  Dtype   
---  ------                                          --------------  -----   
 0   as_built_plan                                   2379 non-null   float64 
 1   available_as_of                                 3869 non-null   category
 2   basement                                        2064 non-null   float64 
 3   bathrooms                                       4476 non-null   float64 
 4   bedroom_1_surface                               3129 non-null   float64 
 5   bedroom_2_surface                               3072 non-null   float64 
 6   bedroom_3_surface                               2592 non-null   float64 
 7   bedrooms                                        4689 non-null   float64 
 8   building_condition                              4436 non-null   category
 9   co2_emission                  

In [39]:
housing_df.state.cat.categories
# Under the hood, pandas is treating the categorical columns as as numbers(codes)

Index(['Brussel', 'Flanders', 'Wallonia'], dtype='object')

In [40]:
# To access the cat codes
housing_df.state.cat.codes

0       1
1       1
2       1
3       1
4       2
       ..
4765    1
4766    1
4767    2
4768    1
4769    2
Length: 4719, dtype: int8

### After converting strings to categories, we can now deal with missing data

In [41]:
# Check missing data
housing_df.isna().sum()

as_built_plan                                     2340
available_as_of                                    850
basement                                          2655
bathrooms                                          243
bedroom_1_surface                                 1590
bedroom_2_surface                                 1647
bedroom_3_surface                                 2127
bedrooms                                            30
building_condition                                 283
co2_emission                                      2802
cadastral_income                                  1068
connection_to_sewer_network                       2724
construction_year                                 1311
covered_parking_spaces                            2018
dining_room                                       3400
double_glazing                                     649
energy_class                                         0
external_reference                                 399
flood_zone

In [42]:
# Check ratio of missing data
housing_df.isna().sum()/len(housing_df)

as_built_plan                                     0.495868
available_as_of                                   0.180123
basement                                          0.562619
bathrooms                                         0.051494
bedroom_1_surface                                 0.336936
bedroom_2_surface                                 0.349015
bedroom_3_surface                                 0.450731
bedrooms                                          0.006357
building_condition                                0.059970
co2_emission                                      0.593770
cadastral_income                                  0.226319
connection_to_sewer_network                       0.577241
construction_year                                 0.277813
covered_parking_spaces                            0.427633
dining_room                                       0.720492
double_glazing                                    0.137529
energy_class                                      0.0000

### Save preprocessed data

In [43]:
# Export housing_df
housing_df.to_csv("housing-df-processed1", index=False)

### Load preprocessed data

In [44]:
df = pd.read_csv("housing-df-processed1", low_memory=False)
df.shape

(4719, 54)

In [45]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,4709,4710,4711,4712,4713,4714,4715,4716,4717,4718
as_built_plan,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,0.0,0.0,1.0,1.0,NaN,0.0,0.0,0.0
available_as_of,After signing the deed,After signing the deed,After signing the deed,NaN,After signing the deed,After signing the deed,Depending on the tenant,Depending on the tenant,After signing the deed,After signing the deed,...,After signing the deed,After signing the deed,NaN,Immediately,NaN,After signing the deed,After signing the deed,NaN,To be defined,NaN
basement,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,NaN,...,1.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN
bathrooms,1.0,1.0,3.0,3.0,NaN,1.0,2.0,2.0,1.0,2.0,...,1.0,3.0,5.0,2.0,3.0,2.0,2.0,5.0,2.0,2.0
bedroom_1_surface,12.0,15.0,22.0,33.0,19.0,13.0,15.0,15.0,NaN,17.0,...,NaN,NaN,NaN,NaN,NaN,11.0,25.0,NaN,NaN,NaN
bedroom_2_surface,10.0,11.0,22.0,30.0,NaN,9.0,15.0,15.0,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,14.0,20.0,NaN,NaN,NaN
bedroom_3_surface,10.0,11.0,22.0,23.0,NaN,NaN,NaN,NaN,NaN,15.0,...,NaN,NaN,NaN,NaN,NaN,15.0,18.0,NaN,NaN,NaN
bedrooms,3.0,4.0,5.0,5.0,1.0,2.0,2.0,2.0,4.0,6.0,...,5.0,4.0,9.0,4.0,6.0,6.0,6.0,9.0,3.0,4.0
building_condition,Good,Good,As new,As new,Just renovated,As new,Good,Good,As new,Good,...,To be done up,Good,Good,Just renovated,Just renovated,Good,To be done up,Good,Good,As new
co2_emission,9802.0,NaN,NaN,NaN,58.0,1029.0,71.0,71.0,NaN,103.0,...,NaN,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


### Fill Missing Values - Numeric

In [46]:
from pandas.api.types import is_numeric_dtype as is_num

In [47]:
# find columns with numeric dtypes
num_col_list = []
for label, content in df.items():
  if is_num(content):
    print(label)
    num_col_list.append(label)

as_built_plan
basement
bathrooms
bedroom_1_surface
bedroom_2_surface
bedroom_3_surface
bedrooms
co2_emission
cadastral_income
connection_to_sewer_network
construction_year
covered_parking_spaces
dining_room
double_glazing
flood_zone_type
furnished
garden_surface
gas_water__electricity
kitchen_surface
living_area
living_room_surface
number_of_frontages
office
outdoor_parking_spaces
planning_permission_obtained
possible_priority_purchase_right
primary_energy_consumption
proceedings_for_breach_of_planning_regulations
street_frontage_width
subdivision_permit
surface_of_the_plot
tv_cable
tenement_building
toilets
width_of_the_lot_on_the_street
yearly_theoretical_total_energy_consumption
postal
lat
lng
price


In [48]:
# since we have 14 non numeric columns, it is right that we have 40 numeric columns
# since df has 54 columns in total
len(num_col_list)

40

In [49]:
# check for which numeric cols have missing values
# fill them with the median

# loop through df
for label, content in df.items():
  # check if the col contains int values
  if is_num(content):
    # check if null values exist in such cols
    if pd.isnull(content).sum():
      # Fill numeric value with median
      df[label] = content.fillna(content.median())

In [50]:
# loop through df
for label, content in df.items():
  # check if the col contains int values
  if is_num(content):
    # check if null values exist in such cols
    if pd.isnull(content).sum():
      print(label)

# Now we get no result because we have filled the null values

In [51]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,4709,4710,4711,4712,4713,4714,4715,4716,4717,4718
as_built_plan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
available_as_of,After signing the deed,After signing the deed,After signing the deed,NaN,After signing the deed,After signing the deed,Depending on the tenant,Depending on the tenant,After signing the deed,After signing the deed,...,After signing the deed,After signing the deed,NaN,Immediately,NaN,After signing the deed,After signing the deed,NaN,To be defined,NaN
basement,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
bathrooms,1.0,1.0,3.0,3.0,1.0,1.0,2.0,2.0,1.0,2.0,...,1.0,3.0,5.0,2.0,3.0,2.0,2.0,5.0,2.0,2.0
bedroom_1_surface,12.0,15.0,22.0,33.0,19.0,13.0,15.0,15.0,16.0,17.0,...,16.0,16.0,16.0,16.0,16.0,11.0,25.0,16.0,16.0,16.0
bedroom_2_surface,10.0,11.0,22.0,30.0,14.0,9.0,15.0,15.0,14.0,16.0,...,14.0,14.0,14.0,14.0,14.0,14.0,20.0,14.0,14.0,14.0
bedroom_3_surface,10.0,11.0,22.0,23.0,14.0,14.0,14.0,14.0,14.0,15.0,...,14.0,14.0,14.0,14.0,14.0,15.0,18.0,14.0,14.0,14.0
bedrooms,3.0,4.0,5.0,5.0,1.0,2.0,2.0,2.0,4.0,6.0,...,5.0,4.0,9.0,4.0,6.0,6.0,6.0,9.0,3.0,4.0
building_condition,Good,Good,As new,As new,Just renovated,As new,Good,Good,As new,Good,...,To be done up,Good,Good,Just renovated,Just renovated,Good,To be done up,Good,Good,As new
co2_emission,9802.0,76.0,76.0,76.0,58.0,1029.0,71.0,71.0,76.0,103.0,...,76.0,80.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,2.0


### Fill Missing Values - categorical/non-num

In [52]:
df.isna().sum(), "Data Types", df.dtypes

(as_built_plan                                        0
 available_as_of                                    850
 basement                                             0
 bathrooms                                            0
 bedroom_1_surface                                    0
 bedroom_2_surface                                    0
 bedroom_3_surface                                    0
 bedrooms                                             0
 building_condition                                 283
 co2_emission                                         0
 cadastral_income                                     0
 connection_to_sewer_network                          0
 construction_year                                    0
 covered_parking_spaces                               0
 dining_room                                          0
 double_glazing                                       0
 energy_class                                         0
 external_reference                             

In [53]:
for label, content in df.items():
  if not is_num(content):
    print(label)

available_as_of
building_condition
energy_class
external_reference
heating_type
kitchen_type
latest_land_use_designation
reference_number_of_the_epc_report
surroundings_type
website
housenumber
street
city
state


In [54]:
# data was saved and reloaded and I noticed datatype changed from categories
# back to object as soon as I reloaded the data
# But it seems cat attributes are still accessible

In [55]:
# checking out the codes we got due to converting string to categories

pd.Categorical(df.state)

['Flanders', 'Flanders', 'Flanders', 'Flanders', 'Wallonia', ..., 'Flanders', 'Flanders', 'Wallonia', 'Flanders', 'Wallonia']
Length: 4719
Categories (3, object): ['Brussel', 'Flanders', 'Wallonia']

In [56]:
# Categorical missing values are represented with -1
# we will deal with this later below
pd.Categorical(df.available_as_of).codes

array([ 0,  0,  0, ..., -1,  4, -1], dtype=int8)

In [57]:
# Turn categorical features to numbers and fill missing
for label, content in df.items():
  if not is_num(content):
    # Turn categories into numbers(cat-code) and add +1
    # +1 is done to change missing values to 0. Missing categorical values is -1 by default
    df[label] = pd.Categorical(content).codes + 1

In [58]:
pd.Categorical(df.available_as_of).codes
# cat missing values fixed

array([1, 1, 1, ..., 0, 5, 0], dtype=int8)

In [59]:
df.info()
# all features now have numeric dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4719 entries, 0 to 4718
Data columns (total 54 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   as_built_plan                                   4719 non-null   float64
 1   available_as_of                                 4719 non-null   int8   
 2   basement                                        4719 non-null   float64
 3   bathrooms                                       4719 non-null   float64
 4   bedroom_1_surface                               4719 non-null   float64
 5   bedroom_2_surface                               4719 non-null   float64
 6   bedroom_3_surface                               4719 non-null   float64
 7   bedrooms                                        4719 non-null   float64
 8   building_condition                              4719 non-null   int8   
 9   co2_emission                             

In [60]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,4709,4710,4711,4712,4713,4714,4715,4716,4717,4718
as_built_plan,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
available_as_of,1.000000,1.000000,1.000000,0.000000e+00,1.000000,1.000000,3.000000,3.000000,1.000000,1.000000,...,1.000000,1.000000,0.000000,4.000000,0.000000,1.000000,1.000000,0.000000,5.000000,0.000000
basement,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
bathrooms,1.000000,1.000000,3.000000,3.000000e+00,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,...,1.000000,3.000000,5.000000,2.000000,3.000000,2.000000,2.000000,5.000000,2.000000,2.000000
bedroom_1_surface,12.000000,15.000000,22.000000,3.300000e+01,19.000000,13.000000,15.000000,15.000000,16.000000,17.000000,...,16.000000,16.000000,16.000000,16.000000,16.000000,11.000000,25.000000,16.000000,16.000000,16.000000
bedroom_2_surface,10.000000,11.000000,22.000000,3.000000e+01,14.000000,9.000000,15.000000,15.000000,14.000000,16.000000,...,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,20.000000,14.000000,14.000000,14.000000
bedroom_3_surface,10.000000,11.000000,22.000000,2.300000e+01,14.000000,14.000000,14.000000,14.000000,14.000000,15.000000,...,14.000000,14.000000,14.000000,14.000000,14.000000,15.000000,18.000000,14.000000,14.000000,14.000000
bedrooms,3.000000,4.000000,5.000000,5.000000e+00,1.000000,2.000000,2.000000,2.000000,4.000000,6.000000,...,5.000000,4.000000,9.000000,4.000000,6.000000,6.000000,6.000000,9.000000,3.000000,4.000000
building_condition,2.000000,2.000000,1.000000,1.000000e+00,3.000000,1.000000,2.000000,2.000000,1.000000,2.000000,...,4.000000,2.000000,2.000000,3.000000,3.000000,2.000000,4.000000,2.000000,2.000000,1.000000
co2_emission,9802.000000,76.000000,76.000000,7.600000e+01,58.000000,1029.000000,71.000000,71.000000,76.000000,103.000000,...,76.000000,80.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,2.000000


In [61]:
df.isna().sum()

as_built_plan                                     0
available_as_of                                   0
basement                                          0
bathrooms                                         0
bedroom_1_surface                                 0
bedroom_2_surface                                 0
bedroom_3_surface                                 0
bedrooms                                          0
building_condition                                0
co2_emission                                      0
cadastral_income                                  0
connection_to_sewer_network                       0
construction_year                                 0
covered_parking_spaces                            0
dining_room                                       0
double_glazing                                    0
energy_class                                      0
external_reference                                0
flood_zone_type                                   0
furnished   

### Dealing with outliers

In [132]:
# View our numeric columns
df[num_col_list]

,as_built_plan,basement,bathrooms,bedroom_1_surface,bedroom_2_surface,bedroom_3_surface,bedrooms,co2_emission,cadastral_income,connection_to_sewer_network,...,surface_of_the_plot,tv_cable,tenement_building,toilets,width_of_the_lot_on_the_street,yearly_theoretical_total_energy_consumption,postal,lat,lng,price
0,0.0,1.0,1.0,12.0,10.0,10.0,3.0,9802.0,991.0,1.0,...,555.0,1.0,0.0,2.0,11.0,55603.5,9600.0,50.743367,3.601306,275000
1,0.0,1.0,1.0,15.0,11.0,11.0,4.0,76.0,931.0,1.0,...,6416.0,1.0,0.0,1.0,11.0,55603.5,8750.0,51.042146,3.302600,430000
2,0.0,1.0,3.0,22.0,22.0,22.0,5.0,76.0,1115.0,1.0,...,130.0,1.0,0.0,4.0,6.0,55603.5,2018.0,51.205030,4.410943,949000
3,0.0,1.0,3.0,33.0,30.0,23.0,5.0,76.0,2391.0,1.0,...,735.0,1.0,0.0,4.0,19.0,55603.5,8000.0,51.206826,3.219186,2200000
4,0.0,1.0,1.0,19.0,14.0,14.0,1.0,58.0,230.0,1.0,...,40.0,1.0,0.0,2.0,5.0,24263.0,1301.0,50.717062,4.577550,195000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4714,1.0,1.0,2.0,11.0,14.0,15.0,6.0,76.0,931.0,1.0,...,900.0,1.0,0.0,4.0,11.0,55603.5,1731.0,50.884192,4.274010,944000
4715,0.0,1.0,2.0,25.0,20.0,18.0,6.0,76.0,931.0,1.0,...,13716.0,1.0,0.0,4.0,11.0,55603.5,1980.0,50.985378,4.466114,960000
4716,0.0,1.0,5.0,16.0,14.0,14.0,9.0,76.0,1885.0,1.0,...,1089.0,1.0,1.0,5.0,11.0,129394.0,7800.0,50.630171,3.779321,750000
4717,0.0,1.0,2.0,16.0,14.0,14.0,3.0,76.0,5318.0,1.0,...,1205.0,1.0,1.0,6.0,9.0,55603.5,2000.0,51.225206,4.399929,925000


In [133]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
as_built_plan,4719.0,0.030515,0.172018,0.000000,0.000000,0.000000,0.000000,1.000000e+00
available_as_of,4719.0,1.440771,1.419539,0.000000,1.000000,1.000000,1.000000,5.000000e+00
basement,4719.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00
bathrooms,4719.0,1.588472,1.090909,1.000000,1.000000,1.000000,2.000000,1.300000e+01
bedroom_1_surface,4719.0,17.562831,22.317220,1.000000,15.000000,16.000000,18.000000,1.492000e+03
bedroom_2_surface,4719.0,15.416402,29.657965,1.000000,13.000000,14.000000,16.000000,1.529000e+03
bedroom_3_surface,4719.0,16.089214,59.442549,1.000000,13.000000,14.000000,14.000000,2.867000e+03
bedrooms,4719.0,3.742530,1.527907,1.000000,3.000000,3.000000,4.000000,2.500000e+01
building_condition,4719.0,2.551812,1.502371,0.000000,2.000000,2.000000,4.000000,6.000000e+00
co2_emission,4719.0,263.034541,1420.005972,1.000000,76.000000,76.000000,76.000000,5.655700e+04


# Fit ML Model
We can now fit a model because our data is fully numeric and contains no missing data.

In [62]:
df.head(20)

,as_built_plan,available_as_of,basement,bathrooms,bedroom_1_surface,bedroom_2_surface,bedroom_3_surface,bedrooms,building_condition,co2_emission,...,width_of_the_lot_on_the_street,yearly_theoretical_total_energy_consumption,housenumber,street,city,postal,state,lat,lng,price
0,0.0,1,1.0,1.0,12.0,10.0,10.0,3.0,2,9802.0,...,11.0,55603.5,128,461,184,9600.0,2,50.743367,3.601306,275000
1,0.0,1,1.0,1.0,15.0,11.0,11.0,4.0,2,76.0,...,11.0,55603.5,179,493,236,8750.0,2,51.042146,3.302600,430000
2,0.0,1,1.0,3.0,22.0,22.0,22.0,5.0,1,76.0,...,6.0,55603.5,54,259,8,2018.0,2,51.205030,4.410943,949000
3,0.0,0,1.0,3.0,33.0,30.0,23.0,5.0,1,76.0,...,19.0,55603.5,208,276,39,8000.0,2,51.206826,3.219186,2200000
4,0.0,1,1.0,1.0,19.0,14.0,14.0,1.0,3,58.0,...,5.0,24263.0,51,411,228,1301.0,3,50.717062,4.577550,195000
5,0.0,1,1.0,1.0,13.0,9.0,14.0,2.0,1,1029.0,...,11.0,23142.0,51,126,36,4260.0,3,50.636098,5.156102,260000
6,0.0,3,1.0,2.0,15.0,15.0,14.0,2.0,2,71.0,...,7.0,15060.0,167,373,104,6990.0,3,50.265875,5.451936,275000
7,0.0,3,1.0,2.0,15.0,15.0,14.0,2.0,2,71.0,...,7.0,15060.0,167,373,104,6990.0,3,50.265875,5.451936,275000
8,0.0,1,1.0,1.0,16.0,14.0,14.0,4.0,1,76.0,...,11.0,81227.0,186,85,162,9860.0,2,50.937065,3.769555,480000
9,0.0,1,1.0,2.0,17.0,16.0,15.0,6.0,2,103.0,...,45.0,112946.0,206,129,143,6900.0,3,50.231363,5.350163,425000


### Split data

In [63]:
# We need to separate our Features from our target
X = df.drop(columns="price")
y = df["price"]

In [64]:
# We need to split our data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Prep function for evaluation

In [65]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [66]:
# Create function to evaluate model on a few diff levels
# OVERFITTING - our model is performing better on the test dataset

def show_scores(model):
  # Make predictions
  train_preds = model.predict(X_train)
  test_preds = model.predict(X_test)
  # Evaluations
  scores = {"Train MAE": mean_absolute_error(y_train, train_preds),
            "Test MAE": mean_absolute_error(y_test, test_preds),
            "Train MSE": mean_squared_error(y_train, train_preds),
            "Test MSE": mean_squared_error(y_test, test_preds),
            "Train R^2": r2_score(y_train, train_preds),
            "Test R^2": r2_score(y_test, test_preds),
            }
  return scores

### Fit several models

In [67]:
# Random Forest algorithm
%%time
# instantiate model

# Set the random seed
np.random.seed(42)

model = RandomForestRegressor()
# Fit Model
model.fit(X_train, y_train)

CPU times: user 7.75 s, sys: 54.8 ms, total: 7.81 s
Wall time: 10.7 s


RandomForestRegressor()

In [68]:
show_scores(model)

{'Train MAE': 43507.38891390728,
 'Test MAE': 124173.22270127118,
 'Train MSE': 9155406720.865572,
 'Test MSE': 100779313100.42279,
 'Train R^2': 0.9722157027270257,
 'Test R^2': 0.7335208827762871}

In [ ]:
# Random Forest achieved a score of 0.73

In [69]:
# KNN algorithm
%%time
# instantiate model-2

# Set the random seed
np.random.seed(42)

model2 = KNeighborsRegressor(n_neighbors=5)
# Fit Model
model2.fit(X_train, y_train)

CPU times: user 3.87 ms, sys: 988 µs, total: 4.85 ms
Wall time: 6.28 ms


KNeighborsRegressor()

In [70]:
show_scores(model2)

{'Train MAE': 161595.49578807945,
 'Test MAE': 189326.62733050846,
 'Train MSE': 118858711335.08716,
 'Test MSE': 208482961697.32523,
 'Train R^2': 0.6392944770339514,
 'Test R^2': 0.44873254361310666}

In [71]:
# Decision Tree Regressor
%%time
# instantiate model-3

# Set the random seed
np.random.seed(42)

model3 = DecisionTreeRegressor()

# Fit Model
model3.fit(X_train, y_train)

CPU times: user 98.1 ms, sys: 0 ns, total: 98.1 ms
Wall time: 100 ms


DecisionTreeRegressor()

In [72]:
show_scores(model3)

{'Train MAE': 0.0,
 'Test MAE': 179807.36864406778,
 'Train MSE': 0.0,
 'Test MSE': 227874630199.08475,
 'Train R^2': 1.0,
 'Test R^2': 0.3974573905596763}

#### Hyperparameter Tuning with RandomizedSearch CV

##### The RF Regressor performs better on the test data, so I will perform hyperparameter tuning on it.

In [73]:
# Create random forest grid
# Different RandomForestClassifier hyperparameters
rf_grid = {"n_estimators": np.arange(10, 100, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
           "max_features": [0.5, 1.0, "sqrt", "auto"],
           "max_samples": [3020]}

In [74]:
# instantiate RF using the randomsearch cv
rscv_rf = RandomizedSearchCV(RandomForestRegressor(n_jobs=-1, random_state=42),
                              param_distributions=rf_grid,
                              n_iter=20,
                              cv=5,
                              verbose=True)

In [75]:
# fit the randomized search CV model
%%time
rscv_rf.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

CPU times: user 2min 1s, sys: 1.11 s, total: 2min 2s
Wall time: 1min 24s


RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
                   n_iter=20,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'max_features': [0.5, 1.0, 'sqrt',
                                                         'auto'],
                                        'max_samples': [3020],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
                   verbose=True)

In [76]:
rscv_rf.score(X_train, y_train)

0.9572904567482219

In [77]:
show_scores(rscv_rf)

{'Train MAE': 55710.73821192053,
 'Test MAE': 123716.71514830508,
 'Train MSE': 14073533531.934126,
 'Test MSE': 97105294169.99304,
 'Train R^2': 0.9572904567482219,
 'Test R^2': 0.7432356673994819}

Quickly use pycaret to compare various regression algorithm performance

In [79]:
from pycaret.regression import *

In [80]:
reg_setup = setup(data=df, target='price', session_id=123)

# Compare regression models
compare_models()

,Description,Value
0,Session id,123
1,Target,price
2,Target type,Regression
3,Original data shape,"(4719, 54)"
4,Transformed data shape,"(4719, 54)"
5,Transformed train set shape,"(3303, 54)"
6,Transformed test set shape,"(1416, 54)"
7,Numeric features,53
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,117310.8636,72685116275.3073,262429.1728,0.7626,0.2860,0.2210,0.4360
et,Extra Trees Regressor,122250.5530,82000676184.7597,277464.5897,0.7359,0.2869,0.2330,2.3770
gbr,Gradient Boosting Regressor,129224.0282,76236125398.0839,270854.4849,0.7345,0.3079,0.2545,1.4510
rf,Random Forest Regressor,125298.7026,84013565297.2429,282246.9337,0.7217,0.2892,0.2351,5.0410
lightgbm,Light Gradient Boosting Machine,125517.7907,90886968327.8921,291950.2905,0.7107,0.2932,0.2281,1.1620
dt,Decision Tree Regressor,168525.1726,150188606176.3668,381553.3922,0.4851,0.3995,0.3117,0.0810
knn,K Neighbors Regressor,200989.8969,208213257971.4097,448411.1092,0.3210,0.4632,0.3889,0.0360
omp,Orthogonal Matching Pursuit,218859.6677,242062861994.1663,482844.2462,0.2570,0.5177,0.4766,0.0250
dummy,Dummy Regressor,300152.8310,321271046846.0453,558791.5685,-0.0039,0.7203,0.8038,0.0210
ada,AdaBoost Regressor,478860.4641,306550809422.4956,551443.9790,-0.0612,0.9571,1.5864,0.4310


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1,
             num_parallel_tree=None, random_state=123, ...)

### Using XGboost Regression Algorithm

In [118]:
# Define the XGBoost model
xgbst = XGBRegressor()
xgbst.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [119]:
show_scores(xgbst)

{'Train MAE': 23186.670651903973,
 'Test MAE': 122815.38567908335,
 'Train MSE': 1032224893.4594059,
 'Test MSE': 96892723118.28532,
 'Train R^2': 0.9968674637657464,
 'Test R^2': 0.7437977445209041}

#### - Hyperperameter Tuning - Using RandomSearch CV on xgboost model

In [120]:
# Create hyperparameter grid
param_grid = {
    'learning_rate': np.linspace(0.01, 0.2, 10),
    'max_depth': range(3, 10),
    'min_child_weight': range(1, 6),
    'subsample': np.linspace(0.5, 1, 6),
    'colsample_bytree': np.linspace(0.5, 1, 6),
    'n_estimators': [100, 200, 300, 400, 500]
}

In [122]:
# Create the RandomizedSearchCV object
%%time
rscv_xgbst = RandomizedSearchCV(xgbst, param_distributions=param_grid, n_iter=20, scoring='neg_mean_squared_error', cv=5, random_state=42)

# Fit the model
rscv_xgbst.fit(X_train, y_train)

CPU times: user 5min 33s, sys: 1.48 s, total: 5min 34s
Wall time: 3min 51s


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                   param_distributions={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'learning_rate': array([0.01      , 0.03111111, 0.05222222, 0.07333333, 0.09444444,
       0.11555556, 0.13666667, 0.15777778, 0.17888889, 0.2       ]),
                                        'max_depth': range(3, 10),
                                        'min_child_weight': range(1, 6),
                                        'n_estimators': [100, 200, 300, 400,
                                                         500],
                                        'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
                   random_state=42, scoring='neg_mean_squared_error')

In [123]:
# Print the best hyperparameters
print("Best Hyperparameters:", rscv_xgbst.best_params_)

Best Hyperparameters: {'subsample': 0.9, 'n_estimators': 500, 'min_child_weight': 2, 'max_depth': 5, 'learning_rate': 0.1366666666666667, 'colsample_bytree': 0.8}


In [126]:
show_scores(rscv_xgbst)

{'Train MAE': 15394.20321812914,
 'Test MAE': 113667.3066230386,
 'Train MSE': 468547590.62784666,
 'Test MSE': 86166974293.1922,
 'Train R^2': 0.998578078949254,
 'Test R^2': 0.7721586054014106}

In [127]:
# Evaluate the model on the test set
test_preds = rscv_xgbst.predict(X_test)
pd.DataFrame(test_preds)

,0
0,566183.625000
1,284524.218750
2,239843.109375
3,385913.687500
4,235943.312500
...,...
939,324649.781250
940,304576.781250
941,500247.156250
942,274904.218750
